In [ ]:
import pandas as pd
import yfinance as yf

#try pandas datareader?
#use yf.pdr_override() if opt in for datareader

In [22]:
dat = yf.Ticker("2YY=F")
dat.info

{'maxAge': 86400,
 'priceHint': 4,
 'previousClose': 3.921,
 'open': 3.917,
 'dayLow': 3.896,
 'dayHigh': 3.929,
 'regularMarketPreviousClose': 3.921,
 'regularMarketOpen': 3.917,
 'regularMarketDayLow': 3.896,
 'regularMarketDayHigh': 3.929,
 'volume': 11,
 'regularMarketVolume': 11,
 'averageVolume': 52,
 'averageVolume10days': 62,
 'averageDailyVolume10Day': 62,
 'bid': 3.817,
 'ask': 3.969,
 'askSize': 18,
 'expireDate': 1743379200,
 'openInterest': 454,
 'fiftyTwoWeekLow': 3.452,
 'fiftyTwoWeekHigh': 5.036,
 'fiftyDayAverage': 4.15308,
 'twoHundredDayAverage': 4.14276,
 'currency': 'USD',
 'tradeable': False,
 'quoteType': 'FUTURE',
 'symbol': '2YY=F',
 'language': 'en-US',
 'region': 'US',
 'typeDisp': 'Futures',
 'quoteSourceName': 'Delayed Quote',
 'triggerable': False,
 'customPriceAlertConfidence': 'NONE',
 'contractSymbol': False,
 'headSymbolAsString': '2YY=F',
 'marketState': 'REGULAR',
 'regularMarketChangePercent': 0.12752141,
 'regularMarketPrice': 3.926,
 'shortName': 

In [ ]:
df = yf.download(['GOOG'], period='1mo')
#use only close price?
#252 is the usual number of trading days
df

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,GOOG,GOOG,GOOG,GOOG,GOOG
Date,,,,,
2025-02-24,180.983810,184.879366,180.674166,183.590841,18734000
2025-02-25,177.168152,180.554293,176.568844,179.949986,20832500
2025-02-26,174.501190,177.877349,173.392453,176.743646,23637200
2025-02-27,170.016312,176.389041,169.558825,175.739787,25930500
2025-02-28,172.024017,172.303697,168.198374,170.106204,30049800
2025-03-03,168.468079,174.800860,167.449235,173.532301,24122000
2025-03-04,172.413574,174.965659,167.349336,167.748890,30711400
2025-03-05,174.790878,175.550008,170.735486,172.123918,18848000
